**About** : This notebook is used to perform inference on validation data

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import gc
import os
import ast
import sys
import cv2
import glob
import json
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
warnings.simplefilter("ignore", UserWarning)

In [ ]:
from params import *

from utils.plots import *
from utils.metrics import *
from utils.logger import Config, save_to_folder
from utils.rle import rle_encode, rle_decode

from inference.tweaking import *
from inference.validation import *
from inference.post_process import *

from data.preparation import prepare_data
from data.dataset import SartoriusDataset
from data.transforms import define_pipelines
from inference.validation import inference_val

## Single xp

### Prepare

In [ ]:
df = prepare_data(fix=False, remove_anomalies=True)

pipelines = define_pipelines("configs/config_aug.py")
dataset = SartoriusDataset(df, transforms=pipelines['val_viz'], precompute_masks=False)

In [ ]:
glob.glob(OUT_PATH + "ens_*")

In [ ]:
EXP_FOLDER = OUT_PATH + "ens_8/"

In [ ]:
EXP_FOLDER = OUT_PATH + "ens_12/"

In [ ]:
results = []
for i in tqdm(range(len(df))):
    id_ = df['id'][i]

    preds_mask = np.load(EXP_FOLDER + f"masks_{id_}.npy")
    preds_boxes = np.load(EXP_FOLDER + f"boxes_{id_}.npy")
    
    results.append((preds_boxes, preds_mask))
    
# if EXP_FOLDER == OUT_PATH + "ens_8/":
#     results = [(r[1], r[0]) for r in results]

### Tweak

In [ ]:
thresholds_mask = [0.45]
thresholds_nms = [0.05, 0.1, 0.15]
thresholds_conf = [np.round(0.05 * i, 2) for i in range(4, 17)]

min_sizes = [0, 50, 100, 150]
# min_sizes = [0]

In [ ]:
thresholds_mask = [0.45]
thresholds_nms = [0.15]
thresholds_conf = [0.65]

min_sizes = [0]
# min_sizes = [35]

In [ ]:
scores_tweak, all_cell_types = tweak_thresholds(
    results,
    dataset,
    thresholds_mask,
    thresholds_nms,
    thresholds_conf,
    min_sizes=min_sizes,
    remove_overlap=True,
    corrupt=True,
)

In [ ]:
best_thresholds_mask, best_thresholds_nms, best_thresholds_conf, best_min_sizes = [], [], [], []
best_scores = []

for c in range(len(CELL_TYPES)):
    print(f' -> Cell type {CELL_TYPES[c]} : ')

    scores_class = scores_tweak[c].mean(-2) 
    idx = np.unravel_index(np.argmax(scores_class, axis=None), scores_class.shape)
    best_score = scores_class[idx]
    best_scores.append(best_score)

    best_thresholds_c = (
        thresholds_mask[idx[0]], thresholds_nms[idx[1]], thresholds_conf[idx[3]], min_sizes[idx[2]]
    )
    best_thresholds_mask.append(best_thresholds_c[0])
    best_thresholds_nms.append(best_thresholds_c[1])
    best_thresholds_conf.append(best_thresholds_c[2])
    best_min_sizes.append(best_thresholds_c[3])

    print(
        f"Best score {best_score:.4f} for thresholds (mask, nms, conf, min_size): {best_thresholds_c}\n"
    )

# weights = [Counter(df_oof['cell_type'])[c] for c in CELL_TYPES]
weights = [Counter(all_cell_types)[c] for c in range(len(CELL_TYPES))]

best_score = np.average(best_scores, weights=weights)

print(f'CV score : {best_score:.4f}')

In [ ]:
best_thresholds_mask, best_thresholds_nms, best_thresholds_conf, best_min_sizes = [], [], [], []
best_scores = []

for c in range(len(CELL_TYPES)):
    print(f' -> Cell type {CELL_TYPES[c]} : ')

    scores_class = scores_tweak[c].mean(-2) 
    idx = np.unravel_index(np.argmax(scores_class, axis=None), scores_class.shape)
    best_score = scores_class[idx]
    best_scores.append(best_score)

    best_thresholds_c = (
        thresholds_mask[idx[0]], thresholds_nms[idx[1]], thresholds_conf[idx[3]], min_sizes[idx[2]]
    )
    best_thresholds_mask.append(best_thresholds_c[0])
    best_thresholds_nms.append(best_thresholds_c[1])
    best_thresholds_conf.append(best_thresholds_c[2])
    best_min_sizes.append(best_thresholds_c[3])

    print(
        f"Best score {best_score:.4f} for thresholds (mask, nms, conf, min_size): {best_thresholds_c}\n"
    )

# weights = [Counter(df_oof['cell_type'])[c] for c in CELL_TYPES]
weights = [Counter(all_cell_types)[c] for c in range(len(CELL_TYPES))]

best_score = np.average(best_scores, weights=weights)

print(f'CV score : {best_score:.4f}')

### Eval

In [ ]:
best_thresholds_mask = 0.45
best_thresholds_nms = [0.1, 0.1, 0.15]
best_thresholds_conf = [0.35, 0.45, 0.7]
best_min_sizes = [0, 150, 75]

In [ ]:
metadata = []
all_scores = [[], [], []]

masks_pred, boxes_pred, cell_types = process_results(
    results,
    best_thresholds_mask,
    best_thresholds_nms,
    best_thresholds_conf,
    best_min_sizes,
    remove_overlap=True,
    corrupt=True
)

scores, scores_per_class = evaluate(
    masks_pred,
    dataset,
    cell_types
)

for masks, boxes, cell_type_pred, img_id, score, cell_type in zip(
    masks_pred, boxes_pred, cell_types, dataset.df['id'].values, scores, dataset.df['cell_type'].values
):
    metadata.append({
        'id': img_id,
        'cell_type': cell_type,
        'cell_type_pred': cell_type_pred,
        'rles': [rle_encode(mask) for mask in masks],
        'boxes': boxes.tolist(),
        'score': score
    })

for i, s in enumerate(scores_per_class):
    all_scores[i] += s
    
df_preds_oof = pd.DataFrame.from_dict(metadata)


In [ ]:
print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

In [ ]:
print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

In [ ]:
print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

In [ ]:
print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

In [ ]:
gc.collect()

In [ ]:
print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

In [ ]:
print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

## Several Exps (not used)

In [ ]:
FOLDERS = [
    OUT_PATH + "ens_7/",
    OUT_PATH + "ens_8/",
    OUT_PATH + "ens_12/",
]

In [ ]:
THRESHOLDS_CONF = [
    [0.35, 0.45, 0.65],
    [0.3, 0.4, 0.65],
    [0.35, 0.4, 0.7]
]
THRESHOLDS_NMS = [
    [0.1, 0.1, 0.05],
    [0.1, 0.1, 0.15],
    [0.1, 0.05, 0.05],
]
THRESHOLDS_MASK = [0.45, 0.45, 0.45]

MIN_SIZES = [0, 0, 0]

In [ ]:
THRESHOLDS_CONF_ENS = [0.3, 0.4, 0.65]
THRESHOLDS_NMS_ENS = [0.1, 0.1, 0.05]

In [ ]:
df = prepare_data(fix=False, remove_anomalies=True)

pipelines = define_pipelines("configs/config_aug.py")

dataset = SartoriusDataset(df, transforms=pipelines['val_viz'], precompute_masks=False)

### PP

In [ ]:
def process_masks(boxes, masks, thresholds_mask, thresholds_nms, thresholds_conf, min_sizes, remove_overlap=True, corrupt=False):
    # Cell type
    cell = np.argmax(np.bincount(boxes[:, 5].astype(int)))

    # Thresholds
    thresh_mask = (
        thresholds_mask if isinstance(thresholds_mask, (float, int))
        else thresholds_mask[cell]
    )
    thresh_nms = (
        thresholds_nms if isinstance(thresholds_nms, (float, int))
        else thresholds_nms[cell]
    )
    thresh_conf = (
        thresholds_conf if isinstance(thresholds_conf, (float, int))
        else thresholds_conf[cell]
    )
    min_size = (
        min_sizes if isinstance(min_sizes, (float, int))
        else min_sizes[cell]
    )

    # Binarize
    if thresh_mask is not None:
        masks = masks > (thresh_mask * 255)

    # Sort by decreasing conf
    order = np.argsort(boxes[:, 4])[::-1]
    masks = masks[order]
    boxes = boxes[order]

    # Remove low confidence
    last = (
        np.argmax(boxes[:, 4] < thresh_conf) if np.min(boxes[:, 4]) < thresh_conf
        else len(boxes)
    )
    masks = masks[:last]
    boxes = boxes[:last]

    # NMS
    if thresh_nms > 0:
        masks, boxes, _ = mask_nms(masks, boxes, thresh_nms)
        
    # Remove small masks
    masks, boxes = remove_small_masks(masks, boxes, min_size=min_size)
    
    # Corrupt
    if corrupt and cell == 1:  # astro
        masks = np.array([corrupt_mask(mask)[0] for mask in masks])

    # Remove overlap
    if remove_overlap:
        masks = remove_overlap_naive(masks)

    return masks, boxes, cell

In [ ]:
from inference.wsf import weighted_boxes_fusion_tracking

In [ ]:
def normalize(boxes, shape=ORIG_SIZE):
    boxes = boxes.copy()
    boxes[:, 0] /= ORIG_SIZE[1]
    boxes[:, 1] /= ORIG_SIZE[0]
    
    boxes[:, 2] /= ORIG_SIZE[1]
    boxes[:, 3] /= ORIG_SIZE[0]
    
    return boxes

In [ ]:
def unnormalize(boxes, shape=ORIG_SIZE):
    boxes = boxes.copy()
    boxes[:, 0] *= ORIG_SIZE[1]
    boxes[:, 1] *= ORIG_SIZE[0]
    
    boxes[:, 2] *= ORIG_SIZE[1]
    boxes[:, 3] *= ORIG_SIZE[0]

    return boxes

In [ ]:
def bbox_to_key(bbox):
    return str(np.round(bbox[:4], 2))

In [ ]:
# Fuse masks that belong to fused boxes
def get_wsf_mask(wbf_box, wbf_org, pmasks, pmasks_lkup, thres=0.5):
    
    w, h = 520, 704
    mask = np.zeros((w, h), dtype=np.uint8)

    for i in range(len(wbf_org)):
        key = bbox_to_key(list(wbf_org[i][4:]))
        model = int(wbf_org[i][3])
        
        try:
            ind = pmasks_lkup[model][key]
        except KeyError:
            print(key)
            print(pmasks_lkup[model])
        mask = mask + pmasks[model][ind]

        
    # convert thres to integer based on number of boxes
    threshold = thres * len(wbf_org)
    mask = mask > threshold
    

    # remove pixels outside WBF box
    m2 = np.zeros((w, h), dtype=np.uint8)
    x1 = max(0, int(h * wbf_box[0]))
    y1 = max(0, int(w * wbf_box[1]))
    x2 = min(h, int(h * wbf_box[2]))
    y2 = min(w, int(w * wbf_box[3]))
    m2[y1:y2, x1:x2] = 1
#     mask = mask * m2

    return mask.astype(np.uint8)

In [ ]:
def wsf_fct(boxes, masks, thres=0.5):
    boxes = [normalize(b) for b in boxes]
    
    mappings = [{bbox_to_key(list(b)): i for i, b in enumerate(bs)} for bs in boxes]

    cells = [np.argmax(np.bincount(b[:, 5].astype(int))) for b in boxes]
    cell = np.argmax(np.bincount(cells))

    # weighted boxes fusion
    wbf_boxes, wbf_scores, _, wbf_originals = weighted_boxes_fusion_tracking(
        [b[:, :4] for b in boxes],
        [b[:, 4] for b in boxes],
        labels_list=[np.ones(len(b)) for b in boxes],
        iou_thr=0.55,
#         skip_box_thr=THRESHOLDS[pred_class],
    )
    wbf_boxes = np.concatenate([wbf_boxes, wbf_scores[:, None], cell * np.ones((len(wbf_boxes), 1))], -1)
    
    # Finally, process masks, making sure there is no overlap
#     res = []
#     used = np.zeros(ORIG_SIZE, dtype=int)
    
#     pred_label = CLASS_LABELS[pred_class + 1]
#     min_key = pred_label + " min"
#     major_axis_len_min = cell_df[cell_df.feature == "major_axis_len"][min_key].iloc[0]
    # process
    new_masks, new_boxes = [], []
    for i in range(len(wbf_boxes)):
        
        if len(wbf_originals[i]) < 1:
#             print('skip')
            continue

        mask = get_wsf_mask(
            wbf_boxes[i], wbf_originals[i], masks, mappings, thres=0.
        )
        
#         plt.imshow(mask)

        # get shape properties
        props = measure.regionprops(mask)

        # if there are multiple separated masks, pick the larger one
        areas = []
        for a in range(len(props)):
            areas.append(props[a].area)
        try:
            target = np.argmax(areas)
        except:
            continue
        # extract properties of interest
        major_axis_len = props[target].major_axis_length
        # check against limits
        if major_axis_len >= major_axis_len_min:
            mask = mask * (1 - used)
            # check if mask is chopped up by previous detections
            if len(measure.find_contours(mask, 0.5, positive_orientation="low")) == 1:
                used += mask
                res.append(rle_encode(mask))
        print(wbf_boxes)

        new_masks.append(mask)
        new_boxes.append(wbf_boxes[i])
        
#         break
        
    return np.array(new_boxes), np.array(new_masks)

In [ ]:
scores = {0: [], 1: [], 2: [], "ens": []}
N = 50

for i in tqdm(range(len(df.head(N)))):
#     i = 2
    id_ = df['id'][i]

    preds_mask = [np.load(f + f"masks_{id_}.npy") for f in FOLDERS]
    preds_boxes = [np.load(f + f"boxes_{id_}.npy") for f in FOLDERS]
    
    data = dataset[i]
    img = data['img']
    truth = data['gt_masks'].masks.copy().astype(int)
#     boxes_truth = data['gt_bboxes']
    rle_truth = dataset.encodings[i].tolist()

    bs = []
    ms = []
    rles = []
    
    for i, (m, b) in enumerate(zip(preds_mask, preds_boxes)):
        if m.shape[-1] == 6:
            b, m = m, b  # swap

        masks, boxes, cell  = process_masks(
            b.copy(), m.copy(), THRESHOLDS_MASK, THRESHOLDS_NMS[i], THRESHOLDS_CONF[i], MIN_SIZES, remove_overlap=True, corrupt=False
        )

        rle_pred = [pycocotools.mask.encode(np.asarray(p, order='F')) for p in masks]
        iou = pycocotools.mask.iou(rle_truth, rle_pred, [0] * len(rle_pred))
        score = iou_map(ious=[iou], verbose=0)

        bs.append(boxes)
        ms.append(masks)
        scores[i].append(score)

#         bs.append(b)
#         ms.append(m)
        rles.append(rle_pred)
        
        print(f' -> IoU mAP #{i}: {score:.4f}')

#         plt.figure(figsize=(15, 10))
#         plot_sample(img, mask=masks.astype(int), boxes=boxes)
#         plt.axis(False)
#         plt.show()
    
    boxes_m, masks_m = wsf_fct(bs, ms)
        
    masks_m, boxes_m, cell  = process_masks(
        boxes_m, masks_m, 0., THRESHOLDS_NMS_ENS, THRESHOLDS_CONF_ENS, MIN_SIZES, remove_overlap=True, corrupt=False
    )

    rle_pred = [pycocotools.mask.encode(np.asarray(p, order='F')) for p in masks_m]
    iou = pycocotools.mask.iou(rle_truth, rle_pred, [0] * len(rle_pred))
    score = iou_map(ious=[iou], verbose=0)
    scores['ens'].append(score)

    print(f' -> IoU mAP ens: {score:.4f}\n')

#     plt.figure(figsize=(15, 10))
#     plot_sample(img, mask=masks_m.astype(int), boxes=boxes_m)
#     plt.axis(False)
#     plt.show()
        
        
#     break

In [ ]:
dfp = df.copy().head(N)
dfp['scores_0'] = scores[0]
dfp['scores_1'] = scores[1]
dfp['scores_2'] = scores[2]
dfp['scores_ens'] = scores["ens"]

dfp[['cell_type', 'scores_0', 'scores_1', 'scores_2', 'scores_ens']].groupby('cell_type').mean()

In [ ]:
fig = plot_preds_iou(
    img,
#     ms[0].astype(int),
    masks_m.astype(int),
    truth,
    plot_tp=True)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
)

fig.show()

In [ ]:
fig = plot_preds_iou(
    img,
    ms[0].astype(int),
#     masks_m.astype(int),
    truth,
    plot_tp=True)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
)

fig.show()

In [ ]:
MIN_SIZE = 0

In [ ]:
THRESHOLDS_CONF = [
#     [0.35, 0.45, 0.65],
    [0.3, 0.4, 0.65],
#     [0.35, 0.4, 0.7]
]
THRESHOLDS_NMS = [
#     [0.1, 0.1, 0.05],
    [0.1, 0.1, 0.15],
#     [0.1, 0.05, 0.05],
]
THRESHOLDS_MASK = [0.45, 0.45, 0.45]

In [ ]:
scores = []
for i in tqdm(range(len(df))[:30]):
    id_ = df['id'][i]

    preds_mask = [np.load(f + f"masks_{id_}.npy") for f in FOLDERS]
    preds_boxes = [np.load(f + f"boxes_{id_}.npy") for f in FOLDERS]
    
#     data = dataset[i]
#     img = data['img']
#     truth = data['gt_masks'].masks.copy().astype(int)
#     boxes_truth = data['gt_bboxes']
    rle_truth = dataset.encodings[i].tolist()

    bs = []
    ms = []
    rles = []
    
    for i, (m, b) in enumerate(zip(preds_mask, preds_boxes)):
        if m.shape[-1] == 6:
            b, m = m, b  # swap
            
        masks, boxes, cell  = process_masks(
            b, m, THRESHOLDS_MASK, THRESHOLDS_NMS[i], THRESHOLDS_CONF[i], remove_overlap=True, corrupt=True
        )
        
#         masks = [m for m in masks if m.sum() > MIN_SIZE]

        rle_pred = [pycocotools.mask.encode(np.asarray(p, order='F')) for p in masks]
        iou = pycocotools.mask.iou(rle_truth, rle_pred, [0] * len(rle_pred))
        score = iou_map(ious=[iou])
        
        scores.append(score)
        bs.append(boxes)
        ms.append(masks)

#         bs.append(b)
#         ms.append(m)
        
#     boxes_m, masks_m = merge_preds_vote(bs, ms)
        
#     masks_m, boxes_m, cell  = process_masks(
#         boxes_m, masks_m, 0., THRESHOLDS_NMS_ENS, THRESHOLDS_CONF_ENS, remove_overlap=True, corrupt=True
#     )

#     rle_pred = [pycocotools.mask.encode(np.asarray(p, order='F')) for p in masks_m]
#     iou = pycocotools.mask.iou(rle_truth, rle_pred, [0] * len(rle_pred))
#     score = iou_map(ious=[iou])

#     scores.append(score)

In [ ]:
df_preds_oof = df.copy().head(len(scores))  # 0
df_preds_oof['score'] = scores

print(f' -> IoU mAP : {df_preds_oof.score.mean():.4f}\n')
df_preds_oof[['cell_type', 'score']].groupby('cell_type').mean()

cort : 75
astro : 125
shsy5y : 50